### Identify projects with activity in every months for analysis period

Filter projects with activity for at least 12 consecutive months:
* 1 closed pull request
* SET prs_selected = TRUE



In [27]:
repos = getProjects()

for repo in repos:    
    project = repo[0]
    analysisPoint = repo[1]
    ci_service = repo[2]
    
    
    if ci_service is not None:
        '''
        previous_period = classifyProjectPR(project,analysisPoint,-1,-12)
        following_period = classifyProjectPR(project,analysisPoint,1,12)

        if previous_period is not None and following_period is not None:
            prs_period = countPRsPeriod(project,previous_period,following_period)
            setRQ2Classification(project,True,24,prs_period,previous_period,following_period)
            setRQ2Classification(project)
            print('PRs: {}  -  Project: {} - Init: {} - Finish: {}'.format(prs_period,project,previous_period,following_period))
        '''
        created = analysisPoint
    else:
        created = getFirstPRDate(project)
    
    end_period = getLastPRDate(project)
    if created is None or end_period is None:
        print('Project: {}  -  created is None or end_period is None'.format(project))
        continue
    
    identifyPRActivity(project,created,end_period)

Months: 20  -  PRs: 129  -  Project: agershun/alasql - Init: 2020-01-02 03:07:50-03:00
Months: 34  -  PRs: 2453  -  Project: algolia/react-instantsearch - Init: 2017-04-04 09:35:07-03:00
Months: 21  -  PRs: 81  -  Project: andOTP/andOTP - Init: 2017-08-07 10:33:15-03:00
Months: 19  -  PRs: 306  -  Project: antvis/L7 - Init: 2019-08-13 06:49:11-03:00
Months: 19  -  PRs: 223  -  Project: apache/incubator-retired-gearpump - Init: 2016-04-27 22:03:55-03:00
Months: 19  -  PRs: 168  -  Project: apache/rocketmq-externals - Init: 2018-06-06 00:36:57-03:00
Months: 21  -  PRs: 88  -  Project: athityakumar/colorls - Init: 2019-12-12 05:57:48-03:00
Months: 19  -  PRs: 201  -  Project: audacity/audacity - Init: 2019-03-14 08:57:53-03:00
Months: 26  -  PRs: 125  -  Project: AutoSpotting/AutoSpotting - Init: 2017-07-03 18:33:50-03:00
Months: 17  -  PRs: 128  -  Project: bakape/meguca - Init: 2017-12-10 18:49:12-03:00
Months: 22  -  PRs: 216  -  Project: binast/binjs-ref - Init: 2018-01-17 12:32:42-03

In [94]:
#getFirstPRDate('81813780/AVLoadingIndicatorView')
getLastPRDate('0xced/XCDYouTubeKit')
#getFirstPRDate('angular/angular.js')
selectFirstPRMonth(project,analysisPoint)

datetime.datetime(2014, 7, 31, 9, 54, 1, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-180, name=None))

In [18]:
def identifyPRActivity(project,init,finish):
    min_months = 12
    
    search_date = analysis_init_date = init
    analysis_finish_date = analysis_finish_tmp = None
    increment=0
    period_max = period = prs_max = total_prs = 0
    
    while search_date <= finish:
        
        prs = selectPRs(project,search_date.month,search_date.year)
        if prs[0]>0:
            period += 1
            total_prs += prs[0]
            
            #inicio de uma nova contagem, armazene o mês inicial
            if period == 1:
                #data da primeira PR do mês inicial
                #analysis_init_tmp = selectFirstPRMonth(project,search_date.month,search_date.year)
                analysis_init_tmp = selectFirstPRMonth(project,search_date)
            if period == min_months:
                analysis_finish_tmp = selectLastPRMonth(project,search_date.month,search_date.year)
        else:    
            if period > period_max:
                period_max = period
                prs_max = total_prs                
                #data da primeira PR do mês inicial
                analysis_init_date = analysis_init_tmp 
                analysis_finish_date = analysis_finish_tmp
                
            period = total_prs = 0
             
        increment +=1   
        search_date = init + relativedelta(months=increment)        
        if search_date.month == finish.month and search_date.year == finish.year:
            search_date = finish
    
    #If finish the data and did not change the values, get the current period and total_prs
    if period > period_max:
        period_max = period
        prs_max = total_prs
        #data da primeira PR do mês inicial
        analysis_init_date = analysis_init_tmp 
        analysis_finish_date = selectLastPRMonth(project,search_date.month,search_date.year)
    
    
    if period_max >= min_months:
        print('Months: {}  -  PRs: {}  -  Project: {} - Init: {}'.format(period_max,prs_max,project,analysis_init_date))
        setPrsClassification(project,True,period_max,prs_max,analysis_init_date,analysis_finish_date)
    else:
        print('------------------------------\nMonths: {}  -  PRs: {}  -  Project: {} - Init: {}'.format(period_max,prs_max,project,analysis_init_date))
        setPrsClassification(project,False,period_max,prs_max,analysis_init_date,analysis_finish_date)

In [87]:
def classifyProjectPR(project,analysisPoint, increment=1, limit=12):
    included = True
        
    months = totalprs = totalissues = 0
    #Check previous months
    while included:
        
        search_date = analysisPoint + relativedelta(months=increment)
        
        prs = selectPRs(project,search_date.month,search_date.year)
        #print('Month: {}  -  Year: {}  -  PRs: {}  -  Project: {}'.format(search_date.month,search_date.year,prs[0],project))
        
        if prs is None or prs[0]==0:            
            return None
        else:
            months += 1
            
        '''
        issues = selectIssues(project,search_date.month,search_date.year)
        print('Month: {}  -  Year: {}  -  Issues: {}  -  Project: {}'.format(search_date.month,search_date.year,issues[0],project))
        if issues is None or issues[0]==0:
            included = False
            break
        '''
        if increment > 0:
            increment+=1
            if increment > limit:                
                return selectLastPRMonth(project,search_date.month,search_date.year)
        else:
            increment-=1
            if increment < limit:
                #return selectFirstPRMonth(project,search_date.month,search_date.year)
                return selectFirstPRMonth(project,search_date)

In [7]:
def selectPRs(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT COUNT(ID) from pullrequests where  
        date_part('month', closed_at) = %s
        AND date_part('year', closed_at) = %s 
        AND project_name like %s"""

    cursor.execute(query, [month,year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return row

In [15]:
def selectFirstPRMonth(project,date):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT closed_at from pullrequests where  
        date_part('month', closed_at) = %s
        AND date_part('year', closed_at) = %s 
        AND project_name like %s
        order by closed_at limit 1;"""
    
    #cursor.execute(query, [date.month,date.year,project])
    
    #query = """SELECT closed_at from pullrequests where  
    #    closed_at >= %s
    #    AND project_name like %s
    #    order by closed_at limit 1;"""

    cursor.execute(query, [date.month,date.year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [9]:
def selectLastPRMonth(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT closed_at from pullrequests where  
        date_part('month', closed_at) = %s
        AND date_part('year', closed_at) = %s 
        AND project_name like %s
        order by closed_at desc limit 1;"""

    cursor.execute(query, [month,year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [10]:
def countPRsPeriod(project,init,finish):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT count(id) from pullrequests  
        WHERE closed_at IS NOT NULL 
        AND closed_at >= %s AND closed_at <= %s 
        AND project_name like %s"""

    cursor.execute(query, [init,finish,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [11]:
def getFirstPRDate(project):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT closed_at from pullrequests where  
        project_name like %s
        order by closed_at limit 1;"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [12]:
def getLastPRDate(project):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT closed_at from pullrequests where  
        project_name like %s AND closed_at is not NULL
        order by closed_at desc limit 1;"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [13]:
def selectIssues(project,month,year):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """select COUNT(ID) from issue where  
        date_part('month', closed_at) = %s
        AND date_part('year', closed_at) = %s 
        AND repo_name like %s"""

    cursor.execute(query, [month,year,project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return row

In [14]:
r= selectIssues('alvarotrigo/fullPage.js',10,2013)
r[0]

0

### Aux Functions

In [3]:
import psycopg2
import os
import requests 
import time
import pytz    
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
from scipy import stats
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [4]:
def connectDB():
    f = open('/home/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    return psycopg2.connect(user = "ci_quality",
                              password = pwd,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "Causal_CI_Quality_v4")

In [22]:
def getProjects():
    query = """SELECT repo_name, analysis_point,ci_service From PROJECTS where prs_mined is true and issues_mined is true and prs_selected is false ORDER BY repo_name"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [20]:
def setPrsClassification(repo_name,classification,period_max,total_prs,analysis_init_date,analysis_finish_date):
    query = """UPDATE  projects 
        set prs_selected = %s,
        qtd_pull_requests_months = %s,
        qtd_pull_request_in_period = %s,
        analysis_init = %s,
        analysis_finish = %s
        WHERE repo_name like %s"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [classification,period_max,total_prs,analysis_init_date,analysis_finish_date,repo_name])
    connection.commit()
    cursor.close()
    connection.close()